In [12]:
%run functions_NoiseStudy.py
%matplotlib 

# global variables
fs = 13
type_plot = ['png', 'svg']

Using matplotlib backend: MacOSX


In [2]:
# actual evaluation time
now = datetime.datetime.now()
today = now.strftime("%Y%m%d")

# output folder
save_dir_plots = 'plots/' + today + '_measurement'
if not os.path.exists(save_dir_plots):
    os.makedirs(save_dir_plots)
    
save_dir_res = 'Results/' + today + '_measurement'
if not os.path.exists(save_dir_res):
    os.makedirs(save_dir_res)

In [3]:
# depth profile microsensor
path_ = '/Volumes/HIS-CAMEL/04measurementData/20201127_Noise-vs-resolution-paper/'
file_ms = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/profiles.txt'

# calibration data 
file_calib = 'Results/20210531_calibration/20210531-1455_calibration_1RoI.hdf5'

# measurement data - images of the optode
file_meas = path_ + 'Klaus_Optode_noise_study_26-11-2020/O2gradient_experiement/'

# image cropping to the region of interest (RoI) that should be analyzed
RoI_op = [[(730, 200), (730, 1250), (1290, 1250), (1290, 200)],   # optode1
          [(1560, 200), (1560, 1250), (2100, 1250), (2100, 200)]] # optode2

# Image resolution - determine conversion factor px -> mm 
# user definition using f.ex. imageJ
px2mm, dpi = image_resolution(px=840.6646, dist_mm=30., inch=1.1811)
print('Image resolution: 1cm equals: {:.2f}px, i.e. {:.0f}dpi'.format(px2mm, dpi))

# ................................................................................
# load depth profile(s) of the microsensir for validation of the depth profile
dic_micro = read_microsensor(file_ms=file_ms, encoding='latin-1')

# load measurement data and crop according to the RoI
dint_red, dint_green, dint_ratio = splitImage(path=file_meas, RoI_op=RoI_op)

Image resolution: 1cm equals: 28.02px, i.e. 712dpi


In [4]:
# one example of one optode
inp = 'optode1, set2'
#input('Which optode and which setting shall be visualized? ') # optode1, set2

### Image blur

##### Option-1: Gaussian blur

In [5]:
# preparation lineprofile
surface = (12.9, 12.) # optode 1, optode2
depth_lp = (-4, 4) # depth for 2D line profile 
depth_op =  (-1.5, 1.) # depth for optode excerpt
pos_lp = (7., 3.) # position of the depth profile within the optode
ls_lw = [0] # line with of the drawn depth profile

# kernel size
kernel='gauss'
ls_kernel = [(1,1), (5, 5), (11, 11), (15, 15), (21, 21), (25, 25), (31, 31), (35, 35),
             (41, 41), (45, 45), (51 ,51), (55, 55), (61, 61), (65, 65), (71, 71), 
             (75, 75), (81, 81)]  

# .................................................................
# additional information for visualization
arg = dict({'curve lw': 1.5, 'vmin':-5, 'vmax': 130, 'lw': 0.0, 'offset ms': 0.2,
           'aspect': 4, 'cmap': plt.cm.inferno, 'vmin op': 0, 'vmax op': 100,
           'figsize': (12, 5), 'fontsize': 11, 'marker': ['o', 'd', 'x', '.'],
           'colors': ['slategrey', 'darkorange', 'forestgreen', 'k']})

Option-1.1: Gauss filter applied to ratiometric intensity

In [15]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Rlp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, px2mm=px2mm, 
                                                ls_lw=ls_lw, int_type='ratio')), 
                  ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Roptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], int_type='ratio')),
                      ls_kernel))

In [16]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Rlp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesR = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Rlp, dO2_optode=dO2_Roptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesR[k] = fig_lp

In [ ]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_ratioIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimagesR.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesR[k].savefig(name_DP + t, dpi=300, transparent=False)

Option-1.2: Gauss filter applied to individual intensity channels red and green

In [21]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Slp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_red, dint_ch2=dint_green,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                int_type='single')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Soptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_red, 
                                        dint_ch2=dint_green, surface=surface, 
                                        depth_min=depth_op[0], depth_max=depth_op[1], 
                                        int_type='single')), ls_kernel))

In [22]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Slp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesS = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Slp, dO2_optode=dO2_Soptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesS[k] = fig_lp

In [ ]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_individualChannels-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimagesS.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesS[k].savefig(name_DP + t, dpi=300, transparent=False)

Option-1.3: Gauss filter applied to normalized ratiometric intensity

In [38]:
%run functions_NoiseStudy.py

In [40]:
# determine O2 concentration for the depth profile (1D) for different image blurs
dO2_Nlp = dict(map(lambda k: 
                  (k, O2_lineprofile_compare_v2(inp=inp, surface=surface, kernel=kernel, 
                                                kshape=k, lp=pos_lp, path_calib=file_calib,
                                                dint_ch1=dint_ratio, dint_ch2=None,
                                                px2mm=px2mm, ls_lw=ls_lw, 
                                                int_type='norm')), ls_kernel))

# determine O2 concentration for different image blurs for an excerpt of the optode in 2D
dO2_Noptode = dict(map(lambda k: 
                      (k, O2blur_optode(kshape=k, inp=inp, path_calib=file_calib, 
                                        px2mm=px2mm, kernel=kernel, dint_ch1=dint_ratio, 
                                        surface=surface, depth_min=depth_op[0], 
                                        depth_max=depth_op[1], int_type='norm')), 
                       ls_kernel))

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.490703   0.479063   0.521122   0.468018   0.532548   0.518451   
-1.444778   0.517635   0.497480   0.482400   0.473227   0.500647   0.519984   
-1.409091   0.497030   0.511138   0.494738   0.508574   0.500909   0.513142   
-1.373405   0.519409   0.501307   0.471064   0.515094   0.490778   0.487358   
-1.337719   0.514412   0.514196   0.510001   0.509855   0.510823   0.533048   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.977579   1.040708   1.047986   0.963060   1.031545   1.038918   
 0.874816   0.990251   1.032209   1.027186   1.026863   1.001372   1.044362   
 0.910502   0.940325   1.002301   0.956929   1.040708   1.010451   1.015058   
 0.946188   1.060209   1.015173   1.054444   0.980612   1.021369   0.961818   
 0.981874   0.948167   1.017984   1.006100   0.998077   0.989205   1.001133   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.509009   0.493669   0.505954   0.478080   0.510662   0.509340   
-1.444778   0.506825   0.496665   0.496104   0.484490   0.507855   0.513765   
-1.409091   0.508467   0.503452   0.488338   0.498913   0.500908   0.509983   
-1.373405   0.514229   0.506495   0.488500   0.510306   0.501226   0.507528   
-1.337719   0.522533   0.505289   0.493388   0.514329   0.509501   0.511513   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.968678   1.022142   1.031170   1.004001   1.018961   1.033352   
 0.874816   0.976001   1.023932   1.017278   1.011713   1.013400   1.028821   
 0.910502   0.985596   1.016377   1.007636   1.014705   1.010901   1.009695   
 0.946188   0.994865   1.012143   1.012817   1.002562   1.005640   0.992035   
 0.981874   0.996123   1.006342   1.012182   0.990308   0.992276   0.990314   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.509503   0.500692   0.501359   0.487378   0.504677   0.506282   
-1.444778   0.510440   0.500529   0.497717   0.491177   0.505339   0.508822   
-1.409091   0.512665   0.501649   0.494974   0.497462   0.505417   0.510092   
-1.373405   0.515786   0.502735   0.494436   0.503658   0.506112   0.510656   
-1.337719   0.519181   0.502828   0.496650   0.507546   0.507565   0.511131   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.975886   1.015520   1.024420   1.008975   1.018915   1.025076   
 0.874816   0.979160   1.016090   1.019500   1.007794   1.013966   1.019093   
 0.910502   0.984529   1.014562   1.014885   1.005434   1.008935   1.011196   
 0.946188   0.989932   1.011223   1.011216   1.001786   1.003559   1.003940   
 0.981874   0.993727   1.008360   1.008592   0.998143   0.999613   1.000276   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.509580   0.501866   0.500497   0.491360   0.503775   0.505531   
-1.444778   0.511364   0.501633   0.498708   0.493778   0.504851   0.507584   
-1.409091   0.513397   0.501749   0.497387   0.497483   0.505703   0.509137   
-1.373405   0.515543   0.501891   0.497318   0.501400   0.506491   0.509964   
-1.337719   0.517328   0.501763   0.498817   0.504246   0.507163   0.510213   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.980520   1.014540   1.023717   1.009104   1.017414   1.022327   
 0.874816   0.981869   1.014181   1.019721   1.007013   1.013540   1.017493   
 0.910502   0.984810   1.013198   1.015908   1.004589   1.009236   1.012170   
 0.946188   0.988153   1.011947   1.012529   1.002242   1.005212   1.007318   
 0.981874   0.990701   1.011326   1.009970   1.000614   1.002387   1.004121   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.509412   0.502052   0.500196   0.494793   0.503359   0.505096   
-1.444778   0.511249   0.501891   0.499790   0.496123   0.504393   0.506515   
-1.409091   0.512903   0.501699   0.499675   0.497866   0.505235   0.507728   
-1.373405   0.514267   0.501491   0.500091   0.499770   0.505942   0.508579   
-1.337719   0.515143   0.501223   0.501139   0.501312   0.506344   0.509028   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.985243   1.014935   1.022634   1.009649   1.015442   1.020167   
 0.874816   0.985099   1.014173   1.019825   1.007442   1.012750   1.016759   
 0.910502   0.985886   1.013633   1.016945   1.005394   1.009880   1.013337   
 0.946188   0.987028   1.013608   1.014306   1.003724   1.007129   1.010144   
 0.981874   0.988107   1.014053   1.012035   1.002533   1.004869   1.007485   

           0.214116   0.249802   0.285488   0.32117

...

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.506407   0.502497   0.502611   0.497806   0.504492   0.504490   
-1.444778   0.507194   0.502894   0.503266   0.497998   0.504545   0.504994   
-1.409091   0.507957   0.503308   0.503948   0.498208   0.504627   0.505491   
-1.373405   0.508666   0.503783   0.504645   0.498450   0.504760   0.505992   
-1.337719   0.509335   0.504301   0.505357   0.498721   0.504950   0.506480   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.990112   1.011193   1.012257   1.001443   1.000435   1.008347   
 0.874816   0.991416   1.012607   1.013516   1.002820   1.002463   1.009798   
 0.910502   0.992548   1.013696   1.014492   1.003893   1.004126   1.010934   
 0.946188   0.993598   1.014526   1.015242   1.004730   1.005528   1.011826   
 0.981874   0.994611   1.015140   1.015842   1.005391   1.006700   1.012556   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.506277   0.502674   0.502711   0.497921   0.504745   0.504460   
-1.444778   0.507024   0.503098   0.503359   0.498119   0.504807   0.504970   
-1.409091   0.507745   0.503545   0.504033   0.498344   0.504913   0.505470   
-1.373405   0.508431   0.504038   0.504717   0.498590   0.505075   0.505965   
-1.337719   0.509102   0.504560   0.505414   0.498858   0.505280   0.506458   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.988697   1.008863   1.010103   0.999206   0.997997   1.005937   
 0.874816   0.990377   1.010619   1.011762   1.000966   1.000394   1.007835   
 0.910502   0.991853   1.012027   1.013111   1.002400   1.002417   1.009402   
 0.946188   0.993202   1.013132   1.014209   1.003567   1.004141   1.010689   
 0.981874   0.994448   1.013976   1.015100   1.004516   1.005610   1.011757   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.506125   0.502966   0.502819   0.498127   0.505119   0.504442   
-1.444778   0.506826   0.503411   0.503464   0.498333   0.505216   0.504956   
-1.409091   0.507510   0.503888   0.504128   0.498572   0.505346   0.505457   
-1.373405   0.508172   0.504396   0.504799   0.498830   0.505534   0.505959   
-1.337719   0.508830   0.504959   0.505487   0.499122   0.505776   0.506469   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.985943   1.004773   1.006341   0.995332   0.993875   1.001818   
 0.874816   0.988200   1.007041   1.008587   0.997666   0.996808   1.004377   
 0.910502   0.990180   1.008927   1.010491   0.999642   0.999366   1.006537   
 0.946188   0.991963   1.010482   1.012092   1.001308   1.001589   1.008386   
 0.981874   0.993589   1.011732   1.013456   1.002709   1.003525   1.009963   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.506064   0.503177   0.502885   0.498282   0.505350   0.504455   
-1.444778   0.506735   0.503631   0.503528   0.498500   0.505475   0.504956   
-1.409091   0.507405   0.504124   0.504197   0.498743   0.505648   0.505464   
-1.373405   0.508077   0.504673   0.504876   0.499034   0.505874   0.505998   
-1.337719   0.508746   0.505284   0.505599   0.499372   0.506158   0.506544   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.983724   1.001708   1.003511   0.992441   0.990862   0.998770   
 0.874816   0.986332   1.004325   1.006137   0.995142   0.994148   1.001723   
 0.910502   0.988659   1.006535   1.008396   0.997467   0.997038   1.004271   
 0.946188   0.990756   1.008381   1.010334   0.999457   0.999593   1.006486   
 0.981874   0.992655   1.009924   1.011996   1.001174   1.001826   1.008407   

           0.214116   0.249802   0.285488   0.32117

           0.000000   0.035686   0.071372   0.107058   0.142744   0.178430   \
-1.480464   0.506034   0.503535   0.503041   0.498549   0.505713   0.504502   
-1.444778   0.506707   0.504054   0.503715   0.498818   0.505914   0.505041   
-1.409091   0.507401   0.504610   0.504407   0.499133   0.506159   0.505605   
-1.373405   0.508121   0.505235   0.505132   0.499503   0.506478   0.506208   
-1.337719   0.508854   0.505922   0.505902   0.499922   0.506859   0.506843   
...              ...        ...        ...        ...        ...        ...   
 0.839130   0.979935   0.996732   0.998880   0.987739   0.986042   0.993837   
 0.874816   0.983062   0.999837   1.002020   0.990952   0.989799   0.997336   
 0.910502   0.985862   1.002524   1.004771   0.993781   0.993163   1.000419   
 0.946188   0.988393   1.004820   1.007180   0.996251   0.996166   1.003137   
 0.981874   0.990686   1.006774   1.009282   0.998423   0.998845   1.005534   

           0.214116   0.249802   0.285488   0.32117

In [ ]:
# VISUALIZE image blur along line profile
# prepare microsensor for joint visualization
df_ms = prepMS_plot(index_lp=dO2_Nlp[ls_kernel[0]]['vertical'][ls_lw[0]].index,
                    dic_micro=dic_micro, offset=arg['offset ms'])

# plot lineprofile and excerpt of optode 
dimagesN = dict()
for k in ls_kernel:
    fig_lp = plotLP(kshape=k, dO2_lp=dO2_Nlp, dO2_optode=dO2_Noptode, df_ms=df_ms, arg=arg,
                    header_ms=['Depth (mm)', 'Intensity'], depth_lp=depth_lp, 
                    s=inp.split(',')[1].strip(), depth=df_ms['Depth (mm)'])
    dimagesN[k] = fig_lp

In [ ]:
# save depth profile
now = datetime.datetime.now() 
name_DP_ = save_dir_plots + '/' + now.strftime("%Y%m%d-%H%M%S") + '_DepthProfile_normalizedIntensity-blurred_' + '-'.join([i.strip() for i in inp.split(',')])

for k in dimagesN.keys():
    name_DP = name_DP_ + '_kernel-' + str(k[0]) + '.'
    for t in type_plot:
        dimagesN[k].savefig(name_DP + t, dpi=300, transparent=False)